<a href="https://colab.research.google.com/github/simulate111/Textual-Data-Analysis-25/blob/main/Exercise%20task%205.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet transformers datasets

In [2]:
import os
import subprocess
import json
from transformers import pipeline
from collections import Counter

In [3]:
!wget http://dl.turkunlp.org/TKO_8964_2023/news-en-2021.jsonl

--2025-01-29 21:25:10--  http://dl.turkunlp.org/TKO_8964_2023/news-en-2021.jsonl
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3385882 (3.2M) [application/octet-stream]
Saving to: ‘news-en-2021.jsonl.4’

news-en-2021.jsonl. 100%[===================>]   3.23M  1.63MB/s    in 2.0s    

2025-01-29 21:25:13 (1.63 MB/s) - ‘news-en-2021.jsonl.4’ saved [3385882/3385882]



In [4]:
#REad 50 articles
news_data = []
with open("news-en-2021.jsonl") as f:
    for i, line in enumerate(f):
        if i >= 50:
            break
        news_data.append(json.loads(line))
texts = [article["text"] for article in news_data if "text" in article]
print("Sample text from article:", texts[0][0:200])

Sample text from article: Finland's government is pushing ahead with plans to introduce a Covid pass, following a meeting of ministers at the House of the Estates in Helsinki on Thursday afternoon. 
 "There are still many open


In [5]:
#Model
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenCla

In [6]:
#get access to raw predictions
raw_predictions= [ner_pipeline(text) for text in texts]
print(raw_predictions[0][0:2])

[{'entity': 'B-LOC', 'score': 0.99978787, 'index': 1, 'word': 'Finland', 'start': 0, 'end': 7}, {'entity': 'B-MISC', 'score': 0.998447, 'index': 13, 'word': 'Co', 'start': 64, 'end': 66}]


In [7]:
#the most common invalid transitions
def invalid_transitions(predictions):
    invalid_transition= Counter()
    for pred in predictions:
        prev_label = None
        for token in pred:
            label = token["entity"]
            if prev_label and label.startswith("I-") and prev_label != label.replace("I-", "B-") and prev_label != label:
              invalid_transition[(prev_label, label)] += 1
            prev_label = label
    return invalid_transition

In [8]:
invalid = invalid_transitions(raw_predictions)
for (prev_label, label), count in invalid.items():
    print(f"{prev_label} ----> {label}:", count)

I-PER ----> I-ORG: 4
I-ORG ----> I-PER: 1
B-LOC ----> I-ORG: 1
I-LOC ----> I-ORG: 3
B-PER ----> I-ORG: 4
I-ORG ----> I-MISC: 5
B-MISC ----> I-LOC: 1
I-LOC ----> I-MISC: 1
B-MISC ----> I-ORG: 6
B-ORG ----> I-LOC: 2
B-ORG ----> I-PER: 1


In [9]:
#aggregation
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")
aggregated_predictions = [ner_pipeline(text) for text in texts]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [10]:
print(aggregated_predictions[0][0:2])

[{'entity_group': 'LOC', 'score': 0.99978787, 'word': 'Finland', 'start': 0, 'end': 7}, {'entity_group': 'MISC', 'score': 0.9487479, 'word': 'Covid', 'start': 64, 'end': 69}]


In [11]:
entity_counter = Counter()
entity_mentions = Counter()
for doc in aggregated_predictions:
    #print(doc)
    for entity in doc:
        #print(entity)
        entity_type = entity["entity_group"]
        entity_name = entity["word"]
        entity_counter[entity_type] += 1
        entity_mentions[entity_name] += 1

In [12]:
#the most common entity type in your data and the most common entities
for entity_type, count in entity_counter.most_common():
    print(f"{entity_type}:", count)

print("\n\n\n")
for entity_name, count in entity_mentions.most_common(5):
    print(f"{entity_name}:", count)

ORG: 358
PER: 324
LOC: 291
MISC: 144




Finland: 93
Finnish: 34
Helsinki: 29
Covid: 19
Yle: 14


1) Read the model page to figure out which datasets were used to train the model, and which entities the model includes.

Model description:
This model is a bert-base-cased model that was fine-tuned on the English version of the standard CoNLL-2003 Named Entity Recognition dataset.

Bert-base-NER is a fine-tuned BERT model that is ready to use for Named Entity Recognition and achieves state-of-the-art performance for the NER task. It has been trained to recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

The training dataset distinguishes between the beginning and continuation of an entity so that if there are back-to-back entities of the same type, the model can output where the second entity begins. As in the dataset, each token will be classified as one of the following classes:

Abbreviation	Description
O	Outside of a named entity
B-MISC	Beginning of a miscellaneous entity right after another miscellaneous entity
I-MISC	Miscellaneous entity
B-PER	Beginning of a person’s name right after another person’s name
I-PER	Person’s name
B-ORG	Beginning of an organization right after another organization
I-ORG	organization
B-LOC	Beginning of a location right after another location
I-LOC	Location

CoNLL-2003 English Dataset Statistics
This dataset was derived from the Reuters corpus which consists of Reuters news stories.